# Демонстрация работы системы извлечения данных из счетов

Этот ноутбук демонстрирует работу системы на реальных примерах, включая использование ML модели для валидации извлеченных данных.


In [ ]:
import sys
sys.path.append('..')

from app.pdf_processor import PDFProcessor
from app.data_extractor import InvoiceDataExtractor
import json
from pathlib import Path


## 1. Инициализация компонентов


In [ ]:
pdf_processor = PDFProcessor()

# Создаем экстрактор с ML валидацией (если модель обучена)
# Попробуем загрузить ML модель
try:
    model_path = Path("../models/invoice_classifier.pkl")
    if model_path.exists():
        print("🔍 Обнаружена обученная ML модель. Используем валидацию с ML.")
        data_extractor = InvoiceDataExtractor(use_ml_validation=True, model_path=str(model_path))
    else:
        print("ℹ️ ML модель не найдена. Используем только regex извлечение.")
        data_extractor = InvoiceDataExtractor(use_ml_validation=False)
except Exception as e:
    print(f"ℹ️ Не удалось загрузить ML модель: {e}. Используем только regex извлечение.")
    data_extractor = InvoiceDataExtractor(use_ml_validation=False)

print("✅ Компоненты инициализированы")


## 2. Пример текста счета (симуляция)


In [ ]:
# Пример текста счета
sample_invoice_text = """
СЧЕТ № 12345
от 15.01.2024

Продавец: ООО "Компания Рога и Копыта"
ИНН: 1234567890
КПП: 123456789
Адрес: г. Москва, ул. Примерная, д. 1

Покупатель: ИП Иванов Иван Иванович
ИНН: 9876543210

Товары и услуги:
1. Услуга консультации - 1 шт. - 10000.00 руб. - 10000.00 руб.
2. Разработка ПО - 1 шт. - 50000.00 руб. - 50000.00 руб.

Итого: 60000.00 руб.
НДС 20%: 12000.00 руб.
К оплате: 72000.00 руб.

Срок оплаты: 10 дней
"""

print("Пример текста счета:")
print(sample_invoice_text)


## 3. Извлечение структурированных данных


In [ ]:
# Извлекаем данные
extracted_data = data_extractor.extract_invoice_data(sample_invoice_text)

# Выводим результат в формате JSON
print("Извлеченные данные:")
print(json.dumps(extracted_data, ensure_ascii=False, indent=2))


## 4. Обработка PDF файла (если доступен)


In [ ]:
# Пример обработки PDF (раскомментируйте, если есть PDF файл)
# pdf_path = "../data/sample_invoice.pdf"
# 
# if Path(pdf_path).exists():
#     print(f"Обрабатываю файл: {pdf_path}")
#     
#     # Извлекаем текст
#     text = pdf_processor.extract_text(pdf_path)
#     print(f"\nИзвлеченный текст (первые 500 символов):")
#     print(text[:500])
#     
#     # Извлекаем структурированные данные
#     data = data_extractor.extract_invoice_data(text)
#     
#     print("\n" + "="*50)
#     print("СТРУКТУРИРОВАННЫЕ ДАННЫЕ:")
#     print("="*50)
#     print(json.dumps(data, ensure_ascii=False, indent=2))
# else:
#     print(f"Файл {pdf_path} не найден. Используйте пример текста выше.")

print("Для обработки реального PDF файла, поместите его в папку data/ и раскомментируйте код выше")


## 5. Анализ извлеченных данных


In [ ]:
print("АНАЛИЗ ИЗВЛЕЧЕННЫХ ДАННЫХ:")
print("="*50)

if extracted_data.get('invoice_number'):
    print(f"✅ Номер счета: {extracted_data['invoice_number']}")
else:
    print("❌ Номер счета не найден")

if extracted_data.get('date'):
    print(f"✅ Дата: {extracted_data['date']}")
else:
    print("❌ Дата не найдена")

if extracted_data.get('seller'):
    print(f"✅ Продавец: {extracted_data['seller']}")
else:
    print("❌ Продавец не найден")

if extracted_data.get('total_amount'):
    print(f"✅ Общая сумма: {extracted_data['total_amount']}")
else:
    print("❌ Общая сумма не найдена")

if extracted_data.get('items'):
    print(f"✅ Найдено товаров/услуг: {len(extracted_data['items'])}")
    for item in extracted_data['items']:
        print(f"   - {item.get('description', 'N/A')}: {item.get('amount', 'N/A')} руб.")
else:
    print("❌ Товары/услуги не найдены")

# Демонстрация ML валидации (если модель загружена)
if data_extractor.use_ml_validation and data_extractor.ml_classifier:
    print("\n" + "="*50)
    print("ML ВАЛИДАЦИЯ ИЗВЛЕЧЕННЫХ ДАННЫХ:")
    print("="*50)
    
    from app.ml_model import InvoiceFieldClassifier
    
    for field, value in extracted_data.items():
        if value and isinstance(value, str):
            try:
                predicted_type, confidence = data_extractor.ml_classifier.predict(value)
                is_valid, val_confidence = data_extractor.ml_classifier.validate_extraction(
                    field, value, sample_invoice_text
                )
                status = "✅" if is_valid else "⚠️"
                print(f"{field:20s}: {value[:40]:<40} -> {predicted_type} (уверенность: {confidence:.2%}) {status}")
            except Exception as e:
                print(f"{field:20s}: {value[:40]:<40} -> Ошибка валидации: {e}")
